In [25]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [26]:
df_all_cols = pd.read_csv("us2022q2a.csv")
df_additional = pd.read_csv("usfirms2022.csv", usecols=["Ticker", "Sector NAICS\nlevel 1", "Name"])
df_all_cols = df_all_cols.join(df_additional.set_index("Ticker"), on="firm")
df = df_all_cols[["revenue", "cogs", "sgae", "otheropexp", "totalassets", "totalliabilities", "originalprice", "sharesoutstanding", "shortdebt", "incometax", "finexp", "extraincome", "adjprice", "q", "firm"]].copy()

In [27]:

# IV: Operating profit growth, book to market value, short debts, EPSP
df.dropna(thresh=8, inplace=True) #8 are the columns that the dataset keeps even if the organisation was not operational + 1
df['qdate'] = pd.PeriodIndex(df['q'], freq="Q")
df.set_index(['firm'], inplace=True)

In [28]:
#Operating profit growth
df['ebit'] = df["revenue"] - df["cogs"] - df["sgae"] - df["otheropexp"]
df['lebit'] = df.groupby(['firm'])['ebit'].shift(4)
lebit_tmp = df["lebit"].replace(0, np.nan)
df["operatingprofitgrowth"] = df["ebit"] / lebit_tmp

#book to maret value
df["bookvalue"] = df["totalassets"] - df["totalliabilities"]
df["marketvalue"] = df["originalprice"] * df["sharesoutstanding"]
bookvalue_tmp = df["bookvalue"].replace(0, np.nan)
df["booktomarketratio"] = df["marketvalue"] / bookvalue_tmp

#short debts
totalassets_tmp = df["totalassets"].replace(0, np.nan)
df["shortfinancialleverage"] = df["shortdebt"] / totalassets_tmp

#EPSP
df['netincome'] = df["revenue"] - df["cogs"] - df["sgae"] - df["otheropexp"] - df["incometax"] - df["finexp"] + df["extraincome"]
df["epsp"] = (df["netincome"] / df["sharesoutstanding"]) / df["originalprice"]

#F1
df['ladjprice'] = df.groupby(['firm'])['adjprice'].shift(4)
df["ccstockreturns"] = np.log(df["adjprice"]) - np.log(df["ladjprice"])
df["f1"] = df["ccstockreturns"].groupby("firm").shift(-1)

variables_interest = ["operatingprofitgrowth", "booktomarketratio", "shortfinancialleverage", "epsp", "f1"]
independent_variables = ["operatingprofitgrowth", "booktomarketratio", "shortfinancialleverage", "epsp"]
dependent_variables = ["f1"]

In [29]:
df = df.groupby("firm").tail(2).groupby("firm").head(1)
df

,revenue,cogs,sgae,otheropexp,totalassets,totalliabilities,originalprice,sharesoutstanding,shortdebt,incometax,...,operatingprofitgrowth,bookvalue,marketvalue,booktomarketratio,shortfinancialleverage,netincome,epsp,ladjprice,ccstockreturns,f1
firm,,,,,,,,,,,,,,,,,,,,,
A,1674000.0,764000.0,5.340000e+05,0.0,1.032700e+07,5.173000e+06,132.3300,300113.377,0.000,36000.0,...,1.146341,5154000.00,3.971400e+07,7.705472,0.000000,2.830000e+05,0.007126,126.034506,0.045405,-0.213296
AA,3293000.0,2181000.0,2.130000e+05,125000.0,1.598800e+07,9.731000e+06,90.0300,185403.032,1000.000,210000.0,...,2.345455,6257000.00,1.669183e+07,2.667706,0.000063,4.690000e+05,0.028098,32.262641,1.022496,0.217886
AAIC,8470.0,4773.0,0.000000e+00,0.0,9.208830e+05,7.027860e+05,3.4700,35016.392,NaN,2287.0,...,0.983768,218097.00,1.215069e+05,0.557123,NaN,-2.701000e+03,-0.022229,4.040000,-0.152090,-0.222528
AAL,8899000.0,0.0,1.062200e+07,0.0,6.740100e+07,7.634100e+07,18.2500,649160.117,2382000.000,-451000.0,...,1.310266,-8940000.00,1.184717e+07,-1.325187,0.035341,-1.635000e+06,-0.138008,23.900000,-0.269713,-0.514447
AAME,51608.0,0.0,4.781200e+04,0.0,3.750310e+05,2.486080e+05,3.1300,20378.576,NaN,954.0,...,-6.939671,126423.00,6.378494e+04,0.504536,NaN,2.842000e+03,0.044556,3.640795,-0.156671,-0.475675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZVIA,38034.0,23413.0,2.327500e+04,8901.0,1.164800e+05,2.261900e+04,4.5700,66686.135,608.000,12.0,...,NaN,93861.00,3.047556e+05,3.246882,0.005220,-1.089800e+04,-0.035760,NaN,NaN,NaN
ZVO,61633.0,39829.0,2.903600e+04,0.0,1.487510e+05,1.374840e+05,0.8201,34054.879,0.000,78.0,...,0.774553,11267.00,2.792841e+04,2.478779,0.000000,-7.437000e+03,-0.266288,4.060000,-1.599512,-0.982014
ZWS,239600.0,137700.0,5.690000e+04,1100.0,1.118600e+06,9.249000e+05,35.4000,125782.456,5600.000,10000.0,...,0.558524,193700.00,4.452699e+06,22.987604,0.005006,3.020000e+04,0.006782,23.259723,0.416636,0.093173


1. Realiza un análisis exploratorio de las variables:

1.1 Calcula matriz de varianza y covarianza, así como matriz de correlación de las variables independientes y la dependiente. Explicar qué es la varianza, covarianza y correlación. Interpreta la matriz de correlación. Tiene que utilizar álgebra matricial y corroborar resultados con funciones de Python. 

Matriz de varianza y covarianza:

In [30]:
#Covariance matrix linear algebra version
df = df.reset_index()
variables = pd.DataFrame(data=df[variables_interest])
variables.replace(np.nan, 0, inplace=True)
x = np.matrix(variables)
xs = (1/variables.count()["f1"]) * x.T * x

m = variables.mean()
m = np.matrix(m).T
ms = m * m.T
ms

cov = pd.DataFrame(data= xs - ms)
cov.columns = variables_interest
cov.index = variables_interest
cov


,operatingprofitgrowth,booktomarketratio,shortfinancialleverage,epsp,f1
operatingprofitgrowth,447.956521,-6.639717,-0.037949,0.009923,-0.329054
booktomarketratio,-6.639717,21416.186927,0.138741,0.016754,-0.297535
shortfinancialleverage,-0.037949,0.138741,0.011715,-0.000910,-0.005546
epsp,0.009923,0.016754,-0.000910,0.013606,0.036281
f1,-0.329054,-0.297535,-0.005546,0.036281,0.458825


In [31]:
#Covariance matrix inbuilt version
variables.cov()

,operatingprofitgrowth,booktomarketratio,shortfinancialleverage,epsp,f1
operatingprofitgrowth,448.081963,-6.641577,-0.037960,0.009926,-0.329146
booktomarketratio,-6.641577,21422.184179,0.138780,0.016759,-0.297618
shortfinancialleverage,-0.037960,0.138780,0.011719,-0.000910,-0.005548
epsp,0.009926,0.016759,-0.000910,0.013610,0.036291
f1,-0.329146,-0.297618,-0.005548,0.036291,0.458953


In [32]:
#Correlation matrix linear algebra version
x = np.matrix(variables.std())
x.T * x
corr = cov / (x.T * x)
corr

,operatingprofitgrowth,booktomarketratio,shortfinancialleverage,epsp,f1
operatingprofitgrowth,0.999720,-0.002143,-0.016561,0.004018,-0.022946
booktomarketratio,-0.002143,0.999720,0.008757,0.000981,-0.003001
shortfinancialleverage,-0.016561,0.008757,0.999720,-0.072062,-0.075629
epsp,0.004018,0.000981,-0.072062,0.999720,0.459065
f1,-0.022946,-0.003001,-0.075629,0.459065,0.999720


In [33]:
#Correlation matrix inbuilt version
variables.corr()

,operatingprofitgrowth,booktomarketratio,shortfinancialleverage,epsp,f1
operatingprofitgrowth,1.000000,-0.002144,-0.016565,0.004019,-0.022952
booktomarketratio,-0.002144,1.000000,0.008759,0.000982,-0.003002
shortfinancialleverage,-0.016565,0.008759,1.000000,-0.072083,-0.075650
epsp,0.004019,0.000982,-0.072083,1.000000,0.459194
f1,-0.022952,-0.003002,-0.075650,0.459194,1.000000


Explicar qué es la varianza, covarianza y correlación:

Varianza: Mide que tanto varían los datos del promedio. Se define matemáticamente por:
$$\small{\sigma^2 = \frac{\displaystyle\sum_{i=1}^{N} (x_i - \bar{x})^2}{N}} $$

Covarianza: Medida de cuanto coincide la desviación del promedio de una variable de la desviación del promedio de otra variable. Se define como:

$$\small{COV(X, Y) = \frac{\displaystyle\sum_{i=1}^{N} (x_i - \bar{x})(y_i - \bar{y})}{N}} $$

Correlación: Es la covarianza de dos variables normalizada (en una medida de [-1, 1]) por medio de la desviación estándar. Se define matematicamente como:
$$\small{\rho(X, Y) = \frac{COV(X, Y)}{SD(X)SD(Y)}} $$

Interpreta la matriz de correlación:

La matriz de correlación nos indica que hay una correlación muy baja entre todas las variables con la excepción de f1 y epsp que tienen una correlación de 45%, detrás de ella está la correlación entre epsp y shortfinancialleverage de -7%.

1.2 Corre pruebas estadísticas para detectar outliers y leverage points. Tiene que utilizar álgebra matricial para las pruebas y explicar claramente cómo funcionan las pruebas. Puede utilizar funciones de Python para corroborar resultados.

In [34]:
variables = pd.DataFrame(data=df[independent_variables])
variables.replace(np.nan, 0, inplace=True)
ones_vector = np.ones((variables.count()["operatingprofitgrowth"], 1))

Leverages:

In [35]:
#Linear algebra version leverages

x = np.matrix(variables)
x = np.c_[ones_vector, x]
h = x * np.linalg.inv(x.T * x) * x.T

leverages = pd.DataFrame(data={"Leverage": np.diag(h)})
leverages = pd.concat([variables, leverages], axis=1, join="inner")

threshold_leverage = 3 * ((len(independent_variables) + 1) / variables.count()["operatingprofitgrowth"])
leverages[leverages['Leverage'] > threshold_leverage]

,operatingprofitgrowth,booktomarketratio,shortfinancialleverage,epsp,Leverage
41,0.580348,0.167253,0.020691,-1.149488,0.026629
57,1.115374,22.794987,0.824053,0.009715,0.015487
81,1.387921,0.571651,0.003806,-1.018040,0.020919
91,0.000000,0.766084,0.673480,-0.095160,0.010151
93,0.959993,10.281528,0.426055,-0.417307,0.006762
...,...,...,...,...,...
3494,-30.111425,0.874405,0.014813,-1.354837,0.037704
3496,121.333333,0.999248,0.013630,0.261558,0.010929
3515,1.645709,0.878431,0.015959,-0.533867,0.005737
3532,366.200000,2.027569,0.077280,-0.037273,0.083707


In [36]:
#Inbuilt version leverages
df.replace(np.nan, 0, inplace=True)
model = sm.OLS(df[dependent_variables], sm.add_constant(df[independent_variables])).fit()
influence = model.get_influence()
leverage = influence.hat_matrix_diag
leverages_sm = pd.DataFrame(data={"Leverage": leverage})
leverages_sm = pd.concat([variables, leverages_sm], axis=1, join="inner")
leverages_sm[leverages_sm['Leverage'] > threshold_leverage]


,operatingprofitgrowth,booktomarketratio,shortfinancialleverage,epsp,Leverage
41,0.580348,0.167253,0.020691,-1.149488,0.026629
57,1.115374,22.794987,0.824053,0.009715,0.015487
81,1.387921,0.571651,0.003806,-1.018040,0.020919
91,0.000000,0.766084,0.673480,-0.095160,0.010151
93,0.959993,10.281528,0.426055,-0.417307,0.006762
...,...,...,...,...,...
3494,-30.111425,0.874405,0.014813,-1.354837,0.037704
3496,121.333333,0.999248,0.013630,0.261558,0.010929
3515,1.645709,0.878431,0.015959,-0.533867,0.005737
3532,366.200000,2.027569,0.077280,-0.037273,0.083707


Outliers:

In [37]:
# outliers linear algebra
threshold_std_residual = 3
predicted_values = model.predict(sm.add_constant(df[independent_variables]))
errors = np.matrix(df[dependent_variables]).T - np.matrix(predicted_values)

squared_errors = np.square(errors)
mse = squared_errors.sum() / (variables.count()["operatingprofitgrowth"]-(len(independent_variables) + 1))
se = np.sqrt(mse * (1 - leverage))
influence_sum = errors / se
std_residuals = pd.DataFrame(influence_sum.T)
std_residuals.columns = ["student_resid"]
std_residuals = pd.concat([df.f1, std_residuals], axis=1)
std_residuals[std_residuals["student_resid"] > threshold_std_residual]

,f1,student_resid
626,-0.505880,15.721043
1490,1.575847,3.600144
1581,-0.315254,5.702127


In [38]:
#Standarized residuals inbuilt
influence_sum_inbuilt = influence.summary_frame()
f1_res = pd.concat([df.f1, influence_sum_inbuilt], axis=1)
std_residuals_inbuilt = f1_res[["f1", "student_resid"]]
std_residuals_inbuilt[std_residuals_inbuilt["student_resid"] > threshold_std_residual]

,f1,student_resid
626,-0.505880,16.293444
1490,1.575847,3.606197
1581,-0.315254,5.727492


¿Cómo funcionan las pruebas?

Leverage: Mide la distancia en X del dato i del promedio de los datos. Al mismo tiempo cuantifica la influencia que tiene la respuesta observada en el valor predicho de y.

Outliers: Miden el residuo de cada dato en unidades de desviación estándar (de esta manera están normalizado)

2. Hace un análisis de multicolinealidad explicando la prueba e implicaciones en el modelo. 

In [39]:
vif_info = pd.DataFrame()
dependent_variables_df = df[dependent_variables]
independent_variables_df = df[independent_variables]
temporal_df = pd.concat([dependent_variables_df, independent_variables_df], axis=1)
vif_info["feature"] = temporal_df.columns
vif_info["value"] = [vif(temporal_df.values, i) for i in range(len(temporal_df.columns))]
vif_info

,feature,value
0,f1,1.343090
1,operatingprofitgrowth,1.002880
2,booktomarketratio,1.000131
3,shortfinancialleverage,1.046630
4,epsp,1.299244


El VIF sirve para medir qué tanto una variable contribuye al error estándar de la regresión. Debido a que un valor conservador de VIF para problemas de multicolinearidad es de 2.5 y ninguno de nuestros valores se acerca a este valor podemos decir que no hay problemas de este tipo. 

3. Propone e implementa soluciones a los problemas de los puntos anteriores para que el modelo sea el más adecuado. 

Podemos usar cook's distance para medir si un dato es influyente

In [40]:
model = sm.OLS(df[dependent_variables], sm.add_constant(df[independent_variables])).fit()
infl = model.get_influence()
cooks = infl.summary_frame()["cooks_d"]
influential_points = cooks[cooks > 1]
influential_points


626    13.320674
Name: cooks_d, dtype: float64

Usamos el valor conservador de que si es mayor a 1 es un valor influyente y que puede ser eliminado. 

In [41]:
df.drop(influential_points.index, inplace=True)

4. Estima e interpreta un modelo de regresión múltiple después de atender los problemas anteriores. Tiene que utilizar álgebra matricial para estimar coeficientes y errores estándar del modelo de regresión, y utilizar funciones de Python para corroborar resultados. 

In [42]:
#Multiple regression linear algebra
variables = pd.DataFrame(data=df[independent_variables])
variables.replace(np.nan, 0, inplace=True)
ones_vector = np.ones((variables.count()["operatingprofitgrowth"], 1))
x = np.matrix(variables)
x = np.c_[ones_vector, x]
y = np.matrix(df[dependent_variables])
bs = np.linalg.inv(x.T * x) * x.T * y
bs = pd.DataFrame(bs)
variables = ["Const"] + independent_variables
variables
bs.index = variables 
bs

,0
Const,-0.351855
operatingprofitgrowth,-0.000823
booktomarketratio,-0.000015
shortfinancialleverage,-0.207410
epsp,3.354295


In [43]:
#standard error linear algebra
variables = pd.DataFrame(data=df[independent_variables])
error = y - x * np.matrix(bs)
varcovarerror = error.T * error
varcovarerror
n = variables.count()["operatingprofitgrowth"]
k = len(independent_variables) + 1
dfresiduals = n - k 
mse = varcovarerror / dfresiduals
mse2 = mse.item() * np.linalg.inv(x.T * x)
se = pd.DataFrame(data=np.sqrt(np.diag(mse2)))
se

,0
0,0.010264
1,0.000459
2,0.000066
3,0.090003
4,0.093984


In [44]:
#Inbuilt multiple regresion and standard errors
model = sm.OLS(df[dependent_variables], sm.add_constant(df[independent_variables])).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     f1   R-squared:                       0.268
Model:                            OLS   Adj. R-squared:                  0.267
Method:                 Least Squares   F-statistic:                     326.1
Date:                Sat, 19 Nov 2022   Prob (F-statistic):          1.86e-239
Time:                        19:36:11   Log-Likelihood:                -3119.8
No. Observations:                3571   AIC:                             6250.
Df Residuals:                    3566   BIC:                             6281.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -0.3519      0.010    -34.279      0.000      -0.372      -0.332
operatingprofitgrowth     -0.0008      0.000     -1.794      0.073      -0.002    7.66e-05
booktomarketratio      -1.526e-05   6.63e-05     -0.230      0.818      -0.000       0.000
shortfinancialleverage    -0.2074      0.090     -2.304      0.021      -0.384      -0.031
epsp                       3.3543      0.094     35.690      0.000       3.170       3.539
==============================================================================
Omnibus:                      690.651   Durbin-Watson:                   1.987
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2670.703
Skew:                          -0.915   Prob(JB):                         0.00
Kurtosis:                       6.821   Cond. No.                     1.45e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.45e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""